In [3]:
%pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 7.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
from pyspark import SparkConf, pandas as ps
from pyspark.sql import SparkSession, functions as sf, Window
from pyspark.sql.functions import col, collect_list
from pyspark.sql.types import *
import psycopg2
import time
import os
import glob

In [5]:
conf = SparkConf().set("spark.driver.memory", "8g")

spark_session = SparkSession\
    .builder\
    .master("local")\
    .config(conf=conf)\
    .appName("ETL") \
    .getOrCreate()

In [11]:
oltp_params = {
    'user': 'admin',
    'password': 'password',
    "driver": "org.postgresql_postgresql-42.7.3.jar"
}

oltp_url = "jdbc:postgresql://rfm-segmentation-oltp-db-1:5432/trans_oltp"

In [10]:
# print(spark_session.sparkContext._jvm.java.sql.DriverManager.getDrivers())
print(spark_session.sparkContext._jsc.sc().listJars())

Vector(spark://5ab2b1b817ac:43743/jars/org.checkerframework_checker-qual-3.42.0.jar, spark://5ab2b1b817ac:43743/jars/org.postgresql_postgresql-42.7.3.jar)


In [14]:
# df = spark_session.read.jdbc(oltp_url, "trans", properties=oltp_params)
# df = spark_session.read.format('jdbc').option('url', oltp_url).option('dbtable', "trans").option('user', 'admin').option('password', 'password').option('driver', "org.postgresql.Driver").load()

df.show()


+----------+----------+--------------------+--------------+-------------+-----------------+----------------+--------------------+-------+
|  trans_id|trans_date|          total_cost|payment_method|         city|       store_type|discount_applied|           promotion|cust_id|
+----------+----------+--------------------+--------------+-------------+-----------------+----------------+--------------------+-------+
|1000299020|2021-07-03|78.20000000000000...|Mobile Payment|        Miami| Department Store|           false|Discount on Selec...|  33877|
|1000342674|2021-10-25|11.60000000000000...|Mobile Payment|San Francisco|      Supermarket|            true|BOGO (Buy One Get...|  33877|
|1000806142|2020-05-25|90.06000000000000...|    Debit Card|      Chicago|   Warehouse Club|            true|                None| 629479|
|1000008294|2020-07-27|90.77000000000000...|          Cash|San Francisco| Department Store|            true|BOGO (Buy One Get...| 143811|
|1000641795|2020-05-22|68.82000000

In [5]:
# df.Product = df.Product.cast(ArrayType(StringType()))

In [102]:
customers = df.select(['Customer_Name', 'Customer_Category']).distinct()

In [103]:
win_spec = Window.orderBy(sf.monotonically_increasing_id())
cust = customers.withColumn("cust_id", sf.row_number().over(win_spec))
# customers.show()

In [162]:
 # = sf.split(df.Product,',')
# print(products)
# prods = [sf.split(p, ',').alias("Product") for p in products]
# print(prods)
tran_prod = df.select("Transaction_ID", sf.explode(sf.split(sf.regexp_replace(df.Product, "^\\[|'+|\\]$",''),',')).alias("Product")).distinct()
tran_prod.count()


2975433

In [106]:
product = tran_prod.select(['Product']).distinct()
win_spec = Window.orderBy(sf.monotonically_increasing_id())
product2 = product.withColumn("prod_id", sf.row_number().over(win_spec))
product2.show(20)

+---------------+-------+
|        Product|prod_id|
+---------------+-------+
|     Lawn Mower|      1|
|        Pickles|      2|
|         Shrimp|      3|
|           Eggs|      4|
|        Ketchup|      5|
|           Tuna|      6|
|          Water|      7|
|      Olive Oil|      8|
|          Broom|      9|
|       Potatoes|     10|
|         Orange|     11|
|    Light Bulbs|     12|
|  Peanut Butter|     13|
|     Trash Bags|     14|
|         Butter|     15|
|         Cereal|     16|
|        Mustard|     17|
|  Ironing Board|     18|
| Cleaning Spray|     19|
|            Tea|     20|
+---------------+-------+
only showing top 20 rows



In [164]:
tran_prod2 = tran_prod.join(other=product2, on='Product', how='left')
tran_prod2.show(20)


+-------------+--------------+-------+
|      Product|Transaction_ID|prod_id|
+-------------+--------------+-------+
|       Salmon|    1000000529|     22|
|       Salmon|    1000000668|     22|
|     Tomatoes|    1000000547|     34|
|       Razors|    1000000998|     35|
|         Iron|    1000000293|     53|
|  Bath Towels|    1000001094|     63|
|        Water|    1000000561|     66|
|          Tea|    1000001102|     77|
|      Carrots|    1000000494|     89|
|        Bread|    1000001146|     93|
| Paper Towels|    1000000057|     96|
|Peanut Butter|    1000000940|    102|
|      Chicken|    1000001099|    103|
|      Dustpan|    1000000626|    108|
|  Garden Hose|    1000000860|    113|
|         Soda|    1000001100|    117|
| Toilet Paper|    1000000611|    122|
|   Toothbrush|    1000000382|    124|
|      Mustard|    1000000921|    129|
|       Butter|    1000000403|    155|
+-------------+--------------+-------+
only showing top 20 rows



In [150]:
tran = df.select("Transaction_ID", "Date", "Customer_Name", "Customer_Category", "Total_Cost", "Payment_Method", "City", "Store_Type", "Discount_Applied", "Promotion")
tran = tran.join(cust, on=["Customer_Name", "Customer_Category"], how='left')
tran = tran.drop('Customer_Name')
tran = tran.drop('Customer_Category')
print(tran.count())

tran.show()

1000000
1000000
+--------------+-------------------+----------+--------------+-------------+-----------------+----------------+--------------------+-------+
|Transaction_ID|               Date|Total_Cost|Payment_Method|         City|       Store_Type|Discount_Applied|           Promotion|cust_id|
+--------------+-------------------+----------+--------------+-------------+-----------------+----------------+--------------------+-------+
|    1000792166|2021-07-24 11:39:12|     36.18|          Cash|      Seattle|         Pharmacy|            true|BOGO (Buy One Get...| 657065|
|    1000000013|2023-04-02 09:24:11|     17.51|Mobile Payment|      Seattle|   Warehouse Club|           false|BOGO (Buy One Get...|  40487|
|    1000792168|2022-12-24 23:42:25|     17.18|   Credit Card|  Los Angeles|  Specialty Store|           false|                None| 572494|
|    1000792157|2021-07-08 04:41:46|     83.19|    Debit Card|  Los Angeles|         Pharmacy|           false|BOGO (Buy One Get...| 60021

In [ ]:
tran.select("Transaction_ID").distinct().filter(

In [151]:
cust.show()
tran.show()
product2.show()
tran_prod2.show()

+--------------------+-----------------+-------+
|       Customer_Name|Customer_Category|cust_id|
+--------------------+-----------------+-------+
|     Michelle Acosta|          Student|      1|
|       Carolyn Brown|   Senior Citizen|      2|
|        Victor Weeks|         Teenager|      3|
|        Lori Pearson|         Teenager|      4|
|           Amy Davis|          Student|      5|
|      Joshua Coleman|      Young Adult|      6|
|      Nathan Russell|   Senior Citizen|      7|
|        Wayne Fields|      Young Adult|      8|
|      Robert Jackson|      Young Adult|      9|
|Samantha Sherman DVM|      Young Adult|     10|
|         Katie Jones|          Student|     11|
|Alexandria Wilkerson|     Professional|     12|
|      Christina Cook|      Middle-Aged|     13|
|          Kent Jones|          Student|     14|
|      Melissa Morgan|          Retiree|     15|
|     Jessica Mullins|      Young Adult|     16|
|   Mr. Jerome Fisher|          Student|     17|
|  Rachael Mcculloug

In [127]:
db_params = {
    'host': 'rfm-segmentation-olap-db-1',
    'database': 'trans_olap',
    'user': 'admin',
    'password': 'password',
    'port': '5432'
}



In [160]:
def insert_from_df(df, table, conn=psycopg2.connect(**db_params)):
    tmp = './tmp'
    df.write.csv(tmp,mode='overwrite')
    tmp_files = glob.glob('./tmp/*.csv')
    print(tmp_files)
    cursor = conn.cursor()

    try:
        for file in tmp_files:
            with open(file, 'r') as f:
                cursor.copy_from(f, table, sep=",", )
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as e:
        [os.remove(file) for file in tmp_files]
        print("Error: %s" % e)
        conn.rollback()
        cursor.close()
        return 1
    print("Table %s copied from dataframe" % table)
    cursor.execute(F"Select * FROM {table} LIMIT 5;")
    cursor.fetchall()
    cursor.close()
    [os.remove(file) for file in tmp_files]


In [165]:
# insert_from_df(df=tran, table="trans")
# insert_from_df(df=cust, table="cust")
# insert_from_df(df=product2, table="product")

insert_from_df(df=tran_prod2, table="trans_prod")


['./tmp/part-00000-4d15cfb3-dae3-4fb9-b7be-0ac83357b642-c000.csv']
Table trans_prod copied from dataframe
